In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import pickle

In [3]:
class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: # from saved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()
        self.Y_data_n = self.Y_data_n.astype(float)


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users

        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))

        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2].astype(float)
            # take mean
            m = np.mean(ratings)
            # print(m)
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
            print("user: " + str(n) + "> rating: "+ str(ratings) + " - y_data_n: " + str(self.Y_data_n[ids, 2]) + " = norm: " + str(ratings - self.mu[n]))


    """
    Khi có dữ liệu mới, cập nhận Utility matrix bằng cách thêm các hàng này vào cuối Utility Matrix. Để cho đơn giản, giả sử rằng không có users hay items mới, cũng không có ratings nào bị thay đổi.
    """
    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        # self.Y_raw_data = np.concatenate((self.Y_raw_data, new_data), axis = 0)
        # self.Y_data_n = self.Y_raw_data.copy()
        # self.n_users = int(np.max(self.Y_raw_data[:, 0])) + 1 
        # self.n_items = int(np.max(self.Y_raw_data[:, 1])) + 1
        # self.normalize_Y()

        self.Y_raw_data = np.concatenate((self.Y_raw_data, new_data), axis = 0)
        self.Y_data_n = self.Y_raw_data.copy().astype(float)
        self.n_users = np.max(self.Y_raw_data[:, 0].astype(int)) + 1
        self.n_items = np.max(self.Y_raw_data[:, 1].astype(int)) + 1
        self.normalize_Y()
        self.updateX()
        self.updateW()


    # Tính giá trị hàm mất mát:
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this 
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L


    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))

    
    # Xác định các items được đánh giá bởi 1 user, và users đã đánh giá 1 item và các ratings tương ứng:
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user user_id, and the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)


    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)


    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id])
        print(self.mu)
        

        for i in items_rated_by_u:
            
            y_pred[i] = y_pred[i] + self.mu[i]
            print(y_pred[i])

        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings

    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE



In [4]:
filename = 'finalized_model_2.sav'
loaded_rs = pickle.load(open(filename, 'rb'))

data = loaded_rs.Y_raw_data

# df = pd.DataFrame(data, columns=['user','item','rating'])
# df

updated_user = [[2502769, 2, 4], [2502769, 5, 5], [2502769, 7, 1], [2502769, 8, 3], [2502769, 9, 3], [2502769, 29, 4], [2502769, 66, 3], [2502769, 67, 5], [2502769, 88, 4], [2502769, 90, 4], [2502769, 104, 3], [2502769, 105, 3]]

loaded_rs.X.shape

(28, 2)

In [5]:
df = pd.DataFrame(loaded_rs.Y_data_n, columns=['user','item','rating'])

loaded_rs.mu
df.item.unique()
df[df["user"] == 908198]

,user,item,rating


In [6]:
predicted_result = loaded_rs.pred_for_user(908198)
predicted_result

[3.64007308 0.         0.         0.         0.         3.18358622
 0.         0.         0.         0.         0.         0.
 0.         3.09893666 2.9142355  3.79009434 0.         0.
 0.         0.         0.         2.97904762 0.         2.78754266
 0.         3.82348308 0.         3.76218022]


[(0, -0.008529885356742843),
 (1, 0.029402282661336696),
 (2, 0.024175719993954563),
 (3, -0.010768204534110818),
 (4, 0.0017359240613761234),
 (5, 0.007011664979216572),
 (6, 0.048621997952318155),
 (7, -0.003931481005616881),
 (8, 0.00017183989894116363),
 (9, -0.008006193061024427),
 (10, -0.03593767115767701),
 (11, 0.005115375651295601),
 (12, 0.0022838325387339584),
 (13, -0.011431043801002728),
 (14, -0.0027575226578998202),
 (15, -0.003945153874234375),
 (16, -0.005236955874908714),
 (17, -0.015299942472342072),
 (18, -0.004825291992398199),
 (19, -0.03156843584142831),
 (20, -0.019965789159548768),
 (21, -0.007793439405721067),
 (22, 0.0001433434218802035),
 (23, -0.0116314999883746),
 (24, -0.014802021950542258),
 (25, -0.0009733901454451178),
 (26, 0.02190313992067666),
 (27, -0.0009631843653711094)]